# Introduction to Robot Intelligence HW 1: Coding Questions

The is the coding potion of Homework 1. These questions are aimed at testing your ability to code up fundamental mathematical operations using Python and the numpy library.

For submission instructions, please see the theory portion of Homework 1.

In [1]:
import numpy as np

## Part 1 : Solve Systems of linear equations

Given n linear equations with at most `n` unknown variables, write a function `solve_system` that can solve an arbitrary system of equations. Your function should output a single vector of values that satisfies the system. You may assume that `n <= 4`.

The system of equations will be provided as a list of strings as seen in `test_eq`. You may also assume that the variables will always be denoted as `a`, `b`, `c`, and `d`.


In [45]:
def solve_system(equations):
  """"
  Takes in a list of strings for each equation. 
  Returns a numpy array with a row for each equation value
  """
  # TODO: Start
  A = np.zeros((4, 4))
  b = np.zeros(4)
  for i in range(len(equations)):
    chars = equations[i].split()
    num, sign, bflag = 1.0, 1.0, False
    for char in chars:
      if bflag:
        b[i] = float(char)
      elif char == "=":
        bflag = True
      elif char in "abcd":
        A[i][ord(char)-97] = num
        num, sign = 1.0, 1.0
      elif char == "-":
        sign = -1.0
      elif char.lstrip("-").isdigit():
        num = sign * float(char)
  
  results = np.linalg.lstsq(A, b, rcond = 1e-10)[0]
  if (not (np.all(abs(np.dot(A, results) - b) < 1e-10 ))):
    raise ValueError("No solution exists.")
  return results.reshape(-1, 1)
  # TODO: End


In [46]:
def test_eq():
  sys_eq = ['2 a + b - 3 c + d = 9',
            '-5 a + 1 b - 4 c + d = -14',
            'a + 2 b - 10 c = -7',
            'a + 2 b = 13']
  results = solve_system(sys_eq)
  expected = np.array([[3],[5],[2],[4]])

  assert(np.all(abs(expected-results) < 1e-10 ))

In [47]:
test_eq()

## Part 2 : Split a dataset into test and train

In supervised learning, the dataset is usually split into a train set (on which the model is trained) and a test set (to evaluate the trained model). This part of the homework requires writing a function `split_into_train_and_test` that takes a dataset and the test-train split ration as input, and provides the data split as an output. The function takes a `random_state` variable as input, which when kept the same outputs precisely the same split for arbitrary runs of the function.

In [ ]:
def split_into_train_and_test(x_all_LF, frac_test=0.5, random_state=None):
    ''' Divide provided array into train and test sets along first dimension
    User can provide random number generator object to ensure reproducibility.
    Args
    ----
    x_all_LF : 2D np.array, shape = (n_total_examples, n_features) (L, F)
        Each row is a feature vector
    frac_test : float, fraction between 0 and 1
        Indicates fraction of all L examples to allocate to the "test" set
        Returned test set will round UP if frac_test * L is not an integer.
        e.g. if L = 10 and frac_test = 0.31, then test set has N=4 examples
    random_state : np.random.RandomState instance or integer or None
        If int, will create RandomState instance with provided value as seed
        If None, defaults to current numpy random number generator np.random.
    Returns
    -------
    x_train_MF : 2D np.array, shape = (n_train_examples, n_features) (M, F)
        Each row is a feature vector
        Should be a separately allocated array, NOT a view of any input array
    x_test_NF : 2D np.array, shape = (n_test_examples, n_features) (N, F)
        Each row is a feature vector
        Should be a separately allocated array, NOT a view of any input array
    Post Condition
    --------------
    This function should be side-effect free. Provided input array x_all_LF
    should not change at all (not be shuffled, etc.)
    Examples
    --------
    >>> x_LF = np.eye(10)
    >>> xcopy_LF = x_LF.copy() # preserve what input was before the call
    >>> train_MF, test_NF = split_into_train_and_test(
    ...     x_LF, frac_test=0.201, random_state=np.random.RandomState(0))
    >>> train_MF.shape
    (7, 10)
    >>> test_NF.shape
    (3, 10)
    >>> print(train_MF)
    [[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
     [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
     [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
     [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
     [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
     [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
     [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]]
    >>> print(test_NF)
    [[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
     [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
     [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]]
    ## Verify that input array did not change due to function call
    >>> np.allclose(x_LF, xcopy_LF)
    True
    References
    ----------
    For more about RandomState, see:
    https://stackoverflow.com/questions/28064634/random-state-pseudo-random-numberin-scikit-learn
    '''
    if random_state is None:
        random_state = np.random
    # TODO: Start
    x_all_LF_copy = x_all_LF.copy()
    random_state.shuffle(x_all_LF_copy)
    num_train = int((1-frac_test) * len(x_all_LF))  # round down for num_train, which automatically rounds up for num_test
    x_train_MF, x_test_NF = x_all_LF_copy[:num_train,:], x_all_LF_copy[num_train:,:]
    return x_train_MF, x_test_NF
    # TODO: End

### Part 3: Solving a Search Problem

In this problem, given a list of N intervals, we want to find the closest non-overlapping interval greater than each interval in the dataset. The function `closest_interval` takes in the list of intervals and return a list of indices corresponding to the index of the closest interval for each element in the list. If a particular interval does not have a closest non-overlapping interval in the given list, return `-1` corresponding to that element in the list.

In [ ]:
def closest_interval(intervals):
    #TODO: Start
    results = np.array([-1 for _ in range(len(intervals))])

    for i in range(len(intervals)):
      for j in range(len(intervals)):
        if intervals[j][0] > intervals[i][1]:
          if (results[i] == -1) or (intervals[j][0] < intervals[results[i]][0]):
            results[i] = j

    return results
    # TODO: End

In [ ]:
intervals = np.array([[1, 4],
                      [2, 5],
                      [8, 9],
                      [6, 8],
                      [9, 10],
                      [3, 4],
                      [7, 9],
                      [5, 7]])

expected_closest_intervals = closest_interval(intervals)

# Evaluate
results = np.array([7, 3, -1, 4, -1, 7, -1, 2])
assert(np.all(abs(expected_closest_intervals-results) < 1e-10 ))